In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.touch_actions import TouchActions
from lxml import html
import os
import json
import numpy as np
import time
import tablib

In [2]:
driver = webdriver.Chrome('./chromedriver')
actions = ActionChains(driver)
url = driver.command_executor._url 
session_id = driver.session_id  
print("url: ", url)
print("Session Id: ", session_id)
# GO TO WEBSITE
driver.get('https://www.crunchbase.com/')
time.sleep(3)

url:  http://127.0.0.1:49377
Session Id:  f719562fd4a0790b57b14a555c8eaec4


In [98]:
class Scrape:
    def __init__(self, driver, company):
        self.driver = driver
        self.company = company.lower()
    def scrape_overview(self, overview_field):
        # CLICK READ MORE
        try: 
            self.driver.find_elements_by_xpath("//a[@class='cb-link cb-display-inline ng-star-inserted']")[0].click()
        except:
            pass
        # FIND CONTENT CARD
        mat_card = self.driver.find_elements_by_xpath("/html[1]/body[1]/chrome[1]/div[1]/mat-sidenav-container[1]/mat-sidenav-content[1]/entity[1]/page-layout[1]/div[2]/div[1]/div[2]/div[1]/div[1]/entity-section[1]/section-layout[1]/mat-card[1]")
        content = mat_card[0].find_elements_by_class_name('section-layout-content')
        # FIND OVERVIEW FIELD CONTENT
        field_content = content[0].text.split('\n')
        field_content = np.array([cont.lower().strip() for cont in field_content])  
        try: # FINDING DESCRIPTION
            description_info = content[0].find_elements_by_tag_name('description-card')[0].text
        except:
            description_info = " "
        try: # FINDING GEOGRAPHIES
            geography_idx = np.where(field_content == self.company)[0][0]
            geographies_info = field_content[geography_idx+2]
        except:
            print("COULDN'T FIND IT")
            geographies_info = " "
            
        # CREATE OVERVIEW DATA OBJECT
        Overview = {"company": self.company, "content": "FOUND"}
        for interest in overview_field:
            try:
                idx = np.where(field_content==interest)[0].item()
                Overview[str(interest)]=field_content[idx+1]
            except:
                Overview[str(interest)]=" "
        Overview['description'] = description_info   
        Overview['geographies'] = geographies_info
        return Overview
    
    def scrape_funding_rounds(self):
        # GET TOTAL FUNDING AMOUNT FIELD
        try:
            funding_mat_card = self.driver.find_elements_by_xpath("//section-layout[@id='section-funding-rounds']")
            funding_field_content = funding_mat_card[0].text.split('\n')
            funding_field_content = np.array([cont.lower().strip() for cont in funding_field_content])
            tfa_idx = np.where(funding_field_content == "total funding amount")[0].item()
            total_funding_amount = funding_field_content[tfa_idx+1]
        except: 
            total_funding_amount = " "
        # GET LATEST FUNDING FIELDS
        try:
            investors= self.driver.find_elements_by_xpath("//section-layout[@id='section-funding-rounds']//tbody//tr")
            latest_inv_info = investors[0].text.split('\n')
            latest_funding = latest_inv_info[0]
            latest_financing = latest_inv_info[1]
            latest_raise = latest_inv_info[3]
        except:
            latest_funding = " "
            latest_financing = " "
            latest_raise = " "
        # CREATE FUNDING_INFO DATA
        funding_info={"total funding amount": total_funding_amount,
                     "latest funding": latest_funding,
                     "latest financing": latest_financing,
                     "latest raise": latest_raise}
        return funding_info
    
    def scrape_investor(self):
        try: 
            investors= self.driver.find_elements_by_xpath("//section-layout[@id='section-investors']//tbody//tr")
            investors_info = ""
            for i in range(len(investors)):
                investors_info += investors[i].text.split('\n')[0] + ", "
        except:
            investors_info = " "
        return investors_info
    
    def organize_content(self, overview, funding, investor):
        # CREATE ACTIVITY DATA
        if overview['acquired by'] != " ":
            activity = "acquired" +', '+overview['operating status']
        else:
            activity = overview['operating status']
        # CREATE MATURITY LEVEL DATA
        maturity_level = ""
        if overview['funding status'] != " ":
            maturity_level +=  overview['funding status']+','
        if overview['last funding type'] != " ":
            maturity_level +=  overview['last funding type']+','
        if overview['ipo status'] != " ":
            maturity_level +=  overview['ipo status']+','
        if overview['company type'] != " ":
            maturity_level +=  overview['company type']+','
        if maturity_level:
            maturity_level = maturity_level[:-1]
        else:
            maturity_level = " "
        # CREATE CONTENT OBJECT
        Content = {
            "Company": self.company,
            "Content": "FOUND",
            "URL": overview['website'],
            "Activity": activity,
            "Description": overview['description'],
            "Maturity Level": maturity_level,
            "Investors": investor,
            "Category": overview['categories'],
            "Sub Category": " ",
            "Tag1": " ",
            "Tag2": " ",
            "Last Funding": funding["latest funding"],
            "Latest Financing": funding["latest financing"],
            "Latest Raise (in MM)": funding["latest raise"],
            "Total Raised (MM)": funding["total funding amount"],
            "Co-Investors": " ",
            "Year Founded": overview["founded date"],
            "Geographies": overview["geographies"],
            "Founder/CEO" : overview["founders"],
            "Founder Email": overview["contact email"],
            "Founder 2": " ",
            "Founder 2 Email": " ",
            "BrainMind Filter": " ",
            "Ecosystem Member": " ",
            "Source": "crunchbase",
            "Number of Employees": overview["number of employees"],
            "Notes": " ",
            "Investors 3": " ",
            "Investors 5": " ",
            "Investors 6": " "
        }
        return Content
    
    {'total funding amount': '$125m', 'latest funding': 'Jul 19, 2017', 
     'latest financing': 'Series C - Brain Corp', 'latest raise': '$114M'}

In [101]:
class CSV_Data:
    def __init__(self, data, filename):
        self.data = data
        self.len_data = len(data)
        self.tab_data = tablib.Dataset()
        self.filename = filename
        self.fields = ['Company', 'Content', 'URL', 'Activity', 'Description', 'Maturity Level', 'Investors', 'Category', 
                        'Sub Category', 'Tag1', 'Tag2', 'Last Funding', 'Latest Financing', 
                       'Latest Raise (in MM)', 'Total Raised (MM)', 'Co-Investors', 'Year Founded',
                       'Geographies', 'Founder/CEO', 'Founder Email', 'Founder 2', 'Founder 2 Email', 'BrainMind Filter',
                       'Ecosystem Member', 'Source', 'Number of Employees', 'Notes', 'Investors 3',
                       'Investors 5', 'Investors 6']
    
    def create_header(self):
        self.tab_data.headers = self.fields
        
    def create_found(self, i):
        row = []
        row.append(self.data[i]["Company"])
        row.append(self.data[i]["Content"])
        for field in self.fields[2:]:
            row.append(self.data[i][field])
        self.tab_data.append(row)
        
    def create_not_found(self, i):
        row = []
        row.append(self.data[i]["Company"])
        row.append(self.data[i]["Content"])
        for field in self.fields[2:]:
            row.append(" ")
        self.tab_data.append(row)
        
    def run(self):
        for i in range(self.len_data):
            if self.data[i]['Content'] == "FOUND":
                self.create_found(i)
            else:
                self.create_not_found(i)
                
    def export_csv(self):
        csv_data = self.tab_data.export('csv')
        f = open(self.filename, "w")
        f.write(csv_data)
        f.close()

In [100]:
txt_filename = "company_txt/company_abe.txt"
csv_filename = "company_csv/company_test.csv"

with open(txt_filename) as f:
    company_list = f.read().split('\n')
overview_field = ['acquired by', 'operating status', 'funding status', 'last funding type', 
                    'company type', 'website' , 'last funding type', 'categories',
                    'ipo status','founded date', 'founders','contact email',
                    'number of employees', 'founders', 'categories']

Content_List = np.array([])
wait_time = 1.5
for idx_company, company_str in enumerate(company_list):
    # SEARCH COMPANY
    searchbar = driver.find_elements_by_xpath("//input[@id='mat-input-0']")
    searchbar[0].send_keys(company_str)
    searchbar[0].clear()
    searchbar[0].send_keys(Keys.RETURN)
    time.sleep(wait_time)
    
    search_list = driver.find_elements_by_xpath("/html[1]/body[1]/chrome[1]/div[1]/mat-sidenav-container[1]/mat-sidenav-content[1]/search[1]/page-layout[1]/div[1]/div[1]/form[1]/div[2]/results[1]/div[1]/div[1]/div[3]/sheet-grid[1]/div[1]/div[1]/grid-body[1]/div[1]/div[1]")
    found = False
    try:
        for company_ele in search_list[0].find_elements_by_tag_name("grid-row"):
            verify_company = company_ele.text.split('\n')[1]
            is_company = verify_company.lower().find(company_str.lower())
            print("CHECKING :", company_ele.text.split('\n')[1])
            print("IS_COMPANY: ", is_company)
            if is_company >= 0:
                print(company_str, " FOUND")
                found = True
                # CLICK FIRST LINK
                hyperlink_str = "//a[@title='" + verify_company + "']"
                driver.find_element_by_xpath(hyperlink_str).click()
                time.sleep(wait_time)
                
                Scrape_Company = Scrape(driver, verify_company)
                overview = Scrape_Company.scrape_overview(overview_field)
                funding = Scrape_Company.scrape_funding_rounds()
                investor = Scrape_Company.scrape_investor()
                Content = Scrape_Company.organize_content(overview, funding, investor)
                break
            else:
                print(company_str, " NOT FOUND")  
                continue
            print("")
        if not found:
            Content = {"Company": company_str, "Content": "NOT FOUND"}
        Content_List = np.append(Content_List, Content)
        print("______________________________________")
    except:
        print("COULDN'T FIND ", company_str)
        print("______________________________________")
    time.sleep(wait_time)

# CREATE CSV DATAFILE
CSV = CSV_Data(data=Content_List, filename=csv_filename) 
CSV.run()
CSV.create_header()
CSV.export_csv()

CHECKING : CloudMedx Inc
IS_COMPANY:  0
CloudMedx Inc  FOUND
______________________________________
CHECKING : CoachAi
IS_COMPANY:  0
CoachAi  FOUND
______________________________________
CHECKING : COCO-MAT
IS_COMPANY:  0
COCO-MAT  FOUND
______________________________________
CHECKING : Cogisen
IS_COMPANY:  0
Cogisen  FOUND
______________________________________


URL - website
Activity - acquired by, operating status
Description - description
Maturity Level - funding status, last funding type, ipo status, company type
Investors - 
Category  - X
Sub Category  - X
Tag1  - X
Tag2  - X

Last Funding - latest_funding
Latest Financing - latest_financing
Latest Raise (in MM) - latest_raise

Total Raised (MM) - total funding amount
Co-Investors  - X
Year Founded  - founded date
Geographies - geographies
Founder/CEO - founders
Email  - contact email
Founder 2 - X
Email  - X
BrainMind Filter - X
Ecosystem Member - X
Source  - X
Number of Employees  - number of employees
Notes  - X
Investors 3  - X
Investors 5  - X
Investors 6 - X